# Public Prize Draft

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Pool Settings

In [2]:
# Yahoo Public Leagues

TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

SKATER_CATEGORIES = {
    'goals': 1/2, 
    'assists': 1, 
    'plus_minus': 1/4, 
    'powerplay_points': 3/4, 
    'shots_on_goal': 1, 
    'hits': 1
}

GOALIE_CATEGORIES = {
    "wins" : 1, 
    "goals_against_average": 1/2,
    "save_percentage": 1/2, 
    "shutouts": 1/10
}

### Yahoo Ranking Data

In [3]:
age = pd.read_csv("../data/info-capfriendly_com.csv")
age = age[["name", "age"]]

yahoo = pd.read_csv("../data/projections-yahoo_com.csv")
yahoo["yahoo_rank"] = (yahoo["ranking_preseason"] + yahoo["ranking_current"]) / 2
yahoo = pd.merge(yahoo, age, how="left", on="name")

yahoo = yahoo[["team", "age", "name", "position", "yahoo_rank"]]

yahoo.head()

,team,age,name,position,yahoo_rank
0,Col,25.0,Nathan MacKinnon,C,1.5
1,Was,35.0,Alex Ovechkin,LW,3.5
2,Edm,23.0,Connor McDavid,C,2.0
3,NYR,29.0,Artemi Panarin,LW,4.0
4,Edm,25.0,Leon Draisaitl,"C,LW",4.0


### Multiple Positions

In [4]:
multi = (
    yahoo
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

multi.head()

,name,position
0,Nathan MacKinnon,C
1,Alex Ovechkin,LW
2,Connor McDavid,C
3,Artemi Panarin,LW
4,Leon Draisaitl,C


### Projection Data

In [5]:
proj = pd.read_csv("../data/projections-generated.csv")

### Goalies

In [6]:
goalies = proj[proj["position"] == "G"].copy()

goalies = (
    goalies
    [["name"] + list(GOALIE_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

for key, value in GOALIE_CATEGORIES.items():
    goalies[key] *= value
    
goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= sum(GOALIE_CATEGORIES.values())
goalies["rollup"] *= 100

goalies = goalies.reset_index()
goalies = goalies[["name", "rollup"]]

goalies.head()

,name,rollup
0,Andrei Vasilevskiy,95.321957
1,Connor Hellebuyck,68.028854
2,Tuukka Rask,69.554808
3,Carter Hart,66.366662
4,Robin Lehner,73.089270


### Skaters

In [7]:
skaters = proj[proj["position"] != "G"].copy()
skaters = (
    skaters
    [["name"] + list(SKATER_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)
     
for key, value in SKATER_CATEGORIES.items():
    skaters[key] *= value
    
skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= sum(SKATER_CATEGORIES.values())
skaters["rollup"] *= 100

skaters = skaters.reset_index()
skaters = skaters[["name", "rollup"]]

skaters.head()

,name,rollup
0,Connor McDavid,67.893909
1,Nathan MacKinnon,72.391571
2,Leon Draisaitl,64.060035
3,Artemi Panarin,55.368554
4,Alex Ovechkin,65.728938


### Merge

In [8]:
rollup = pd.concat([skaters, goalies])
rollup = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)
rollup.head()

,name,position,rollup
284,Andrei Vasilevskiy,G,95.321957
293,Anton Khudobin,G,75.730069
287,Robin Lehner,G,73.089270
0,Nathan MacKinnon,C,72.391571
307,Philipp Grubauer,G,72.234707


### VORP

In [9]:
vorp = rollup.copy()
vorp["vorp"] = vorp["rollup"]

for position, slots in ROSTER.items():
    replacement = (
        vorp[vorp['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * TEAMS)
        ['vorp']
        .mean()
    )
    vorp.loc[vorp['position'] == position, 'vorp'] = vorp['vorp'] - replacement

vorp.head()

,name,position,rollup,vorp
284,Andrei Vasilevskiy,G,95.321957,35.155361
293,Anton Khudobin,G,75.730069,15.563473
287,Robin Lehner,G,73.089270,12.922674
0,Nathan MacKinnon,C,72.391571,22.250930
307,Philipp Grubauer,G,72.234707,12.068112


### Lineups

In [10]:
lineups = pd.read_csv("../data/lineups-dailyfaceoff_com.csv")
lineups["name"] = lineups["name"].replace("Nathan Mackinnon", "Nathan MacKinnon")

### Draft

In [11]:
draft = pd.merge(yahoo, vorp, how="left", on="name", suffixes=("_yahoo", ""))
draft = pd.merge(draft, lineups, how="left", on="name")

draft = draft.dropna(subset=["vorp"])
draft = draft.groupby("name").head(1)

draft = draft[[
    'team', 'age', 'name', 
    'line', "ppu",
    'position_yahoo', 'position',
    'rollup', 'vorp', 'yahoo_rank',
]]

draft["rollup"] = draft["rollup"].round(1)
draft["vorp"] = draft["vorp"].round(1)
draft['rp'] = draft.groupby("position")['yahoo_rank'].rank(ascending=True)
draft['rp_vorp'] = draft.groupby("position")['vorp'].rank(ascending=False)
draft["rp_arbitrage"] =  draft['rp'] - draft['rp_vorp']
draft['round'] = draft["yahoo_rank"] // TEAMS + 1
draft = draft.sort_values("yahoo_rank", ascending=True)

### Targets

In [13]:
draft["target"] = (
    (draft["age"] <= 30) &
    (draft["rp_arbitrage"] >= -1) &
    (draft["rp_vorp"] <= 24) & 
    (draft["vorp"] >= -5)
)

### Export

In [14]:
draft.to_csv("../data/draft-$500.csv", index=False)